##### The cell below is for you to keep track of the libraries used and install those libraries quickly
##### Ensure that the proper library names are used and the syntax of `%pip install PACKAGE_NAME` is followed

In [2]:
!pip uninstall scikit-learn --yes
!pip uninstall imblearn --yes
!pip install scikit-learn==1.2.2
!pip install imblearn

Found existing installation: scikit-learn 1.4.0
Uninstalling scikit-learn-1.4.0:
  Successfully uninstalled scikit-learn-1.4.0


     ---------------------------------------- 0.0/8.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/8.3 MB 660.6 kB/s eta 0:00:13
     - -------------------------------------- 0.3/8.3 MB 3.2 MB/s eta 0:00:03
     --- ------------------------------------ 0.7/8.3 MB 5.3 MB/s eta 0:00:02
     ----- ---------------------------------- 1.1/8.3 MB 6.3 MB/s eta 0:00:02
     ------- -------------------------------- 1.6/8.3 MB 7.6 MB/s eta 0:00:01
     ---------- ----------------------------- 2.1/8.3 MB 8.2 MB/s eta 0:00:01
     ------------ --------------------------- 2.6/8.3 MB 9.2 MB/s eta 0:00:01
     --------------- ------------------------ 3.2/8.3 MB 9.7 MB/s eta 0:00:01
     ----------------- ---------------------- 3.7/8.3 MB 9.9 MB/s eta 0:00:01
     -------------------- ------------------- 4.3/8.3 MB 10.5 MB/s eta 0:00:01
     ----------------------- ---------------- 4.9/8.3 MB 10.4 MB/s eta 0:00:01
     -------------------------- ------------- 5.5/8.3 MB 10.9 MB/s 

In [3]:
%pip install pandas
%pip install pyarrow
%pip install numpy
%pip install scikit-learn
%pip install imbalanced-learn
%pip install matplotlib



import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## **DO NOT CHANGE** the filepath variable
##### Instead, create a folder named 'data' in your current working directory and 
##### have the .parquet file inside that. A relative path *must* be used when loading data into pandas

In [8]:
# Can have as many cells as you want for code
import pandas as pd
filepath = "./data/datathon.parquet" 
# the initialised filepath MUST be a relative path to a folder named data that contains the parquet file

### **ALL** Code for machine learning and dataset analysis should be entered below. 
##### Ensure that your code is clear and readable.
##### Comments and Markdown notes are advised to direct attention to pieces of code you deem useful.

## Data cleaning

### Deleting unnecessary and combining similar variables



In [4]:
df_orig = pd.read_parquet('catB_train.parquet')
df = df_orig.copy()

exclude = ["grp", "ltc", "gi", "ape", "sumin", "prempaid", "inv",
           "consent", "valid", "ctry", "pr",
           "mindef", "elx", "retail", "housewife",
           "mail", "hold", "hh_20", "pop_20", "hh_size",
           "affconnect", "n_months_since_visit_affcon", "clmcon_visit_days",
           "claim", "index", "recency", "class", "type", "clntnum", "at_least", "occ"]

df1 = df.loc[:, df.apply(lambda x: len(x.unique()) > 1)]

exclude_cols = [col for col in df1.columns if any(substring in col for substring in exclude)]
df1 = df1.drop(columns=exclude_cols)

df1 = df1.loc[:, ~df1.columns.str.contains("last_bought") | df1.columns.str.contains("last_bought_lh")]
df1 = df1.loc[:, ~df1.columns.str.contains("ever_bought") | df1.columns.str.contains("ever_bought_lh")]
df1 = df1.loc[:, ~df1.columns.str.contains("lapse") | df1.columns.str.contains("flg_latest_being_lapse")]


df_everbought = df1.loc[:, df1.columns.str.contains("ever_bought_lh")]
df_everbought['ever_bought_lh_sum'] = df_everbought.filter(like='lh').sum(axis=1)
df_everbought = df_everbought[['ever_bought_lh_sum']]

df_lastbought_lh = df1.filter(like="last_bought_lh")
df_lastbought_lh['last_bought_lh_min'] = df_lastbought_lh.apply(lambda row: min(row.dropna(), default=np.nan), axis=1)
df_lastbought_lh['last_bought_lh_min'] = df_lastbought_lh['last_bought_lh_min'].replace('9999', '0', regex = True)
df_lastbought_lh = df_lastbought_lh[['last_bought_lh_min']]

columns_to_drop = df1.columns[df1.columns.str.contains('ever_bought|last_bought')]
df2 = df1.drop(columns=columns_to_drop, errors='ignore')
df2 = pd.concat([df2, df_lastbought_lh], axis=1, join='inner')
df2 = pd.concat([df2, df_everbought], axis=1, join='inner')

C:\Users\User\AppData\Local\Temp\ipykernel_29200\2503592472.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_everbought['ever_bought_lh_sum'] = df_everbought.filter(like='lh').sum(axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_29200\2503592472.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lastbought_lh['last_bought_lh_min'] = df_lastbought_lh.apply(lambda row: min(row.dropna(), default=np.nan), axis=1)
C:\Users\User\AppData\Local\Temp\ipykernel_29200\2503592472.py:27: SettingWithCopyWar

1. Delete columns with all entries the same (no differentiating factors).
2. Delete columns with headers containing ‘grp’, ‘ltc’, ‘inv’, and ‘gi’ (group, long term care, investment, general) which are unrelated to health and life insurance.
3. Delete columns with headers containing ‘elx’, ‘mindef’, and ‘retail’ where specific programs or sectors contribute little to chances of one buying insurance. 
4. Delete columns related to contact information (containing ‘mail’, ‘email’, ‘call’, ‘sms’, etc.) that have low association with one buying insurance.  
5. Delete columns we deem as irrelevant, i.e. ‘ctrycode’ and ‘is_sg_pr‘’. 

6. Combine the five columns that begin with ‘n_months_last_bought_lh_<product_code>,' representing five different life and health insurance products, into a single column named 'last_bought_lh_min.' Use the minimum value across all products as the indicator of the recent purchase history of the client.
7. Combine the five columns that begin with ‘f_ever_bought_lh_<product_code>,' representing five different life and health insurance products, into a single column named 'ever_bought_lh_sum.' Summing the values across all products as the indicator of the total number of purchases of the client.



### Changing the date of birthe variable to age(categorical)

In [5]:
df3 = df2.copy().drop('f_purchase_lh', axis = 1)
df3['f_ever_declined_la'] = df3['f_ever_declined_la'].fillna(0)
columns_with_missing_values = df3.columns[df3.isnull().any()]

def calculate_age(date_of_birth):
    # Parse the date of birth string into a datetime object
    dob_date = datetime.strptime(date_of_birth, '%Y-%m-%d')

    # Get the current date
    current_date = datetime.now()

    # Calculate the age
    age = current_date.year - dob_date.year - ((current_date.month, current_date.day) < (dob_date.month, dob_date.day))

    return age

df3['cltdob_fix'] = df3['cltdob_fix'].apply(lambda x: calculate_age(x) if x != 'None' else None)

bins = [0, 25, 60, float('inf')]
labels = [0, 1, 2]

# Create a new column 'age_range' based on the age ranges
df3['cltdob_fix'] = pd.cut(df3['cltdob_fix'], bins=bins, labels=labels, right=False)

df3 = df3.rename(columns={'cltdob_fix': 'age'})

def fill_missing_randomly(column):
    # Calculate the percentage of each value in the column
    value_counts = column.value_counts(normalize=True)

    # Replace missing values with values based on the current distribution
    missing_mask = column.isnull()
    column[missing_mask] = np.random.choice(value_counts.index, size=missing_mask.sum(), p=value_counts.values)

for col in columns_with_missing_values:
  fill_missing_randomly(df3[col])

C:\Users\User\AppData\Local\Temp\ipykernel_29200\314704808.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[missing_mask] = np.random.choice(value_counts.index, size=missing_mask.sum(), p=value_counts.values)
C:\Users\User\AppData\Local\Temp\ipykernel_29200\314704808.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  column[missing_mask] = np.random.choice(value_counts.index, size=missing_mask.sum(), p=value_counts.values)
C:\Users\User\AppData\Local\Temp\ipykernel_29200\314704808.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the docum

1. Change the ‘cltdob_fix’ column to ‘age’ column. Convert the client's date of birth to their age and classify it into three categories: below 25 ->0; 25 to 60 -> 1; and above 60 -> 2.

### Changing all the categorial data into integer representation.

In [7]:
label_encoder = LabelEncoder()

df3['annual_income_est'] = label_encoder.fit_transform(df3['annual_income_est'])
df3['stat_flag'] = label_encoder.fit_transform(df3['stat_flag'])
df3['age'] = label_encoder.fit_transform(df3['age'])
df3['race_desc'] = label_encoder.fit_transform(df3['race_desc'])
df3['cltsex_fix'] = label_encoder.fit_transform(df3['cltsex_fix'])

df3['f_purchase_lh'] = df2['f_purchase_lh']
df3['f_purchase_lh'] = df3['f_purchase_lh'].fillna(0)

df3 = df3.astype(int)

1. Change the ‘cltsex_fix’ column to binary expression. I.e Male = 0, Female = 1
2. Change the ‘annual_income_est’ ranges to numbers: ‘Below 30k’ -> 0; ‘30k-60k’ -> 1; ‘60k - 100k’ -> 2;  ‘100k-200k’ -> 3;  ‘Above 200k’ -> 4.
3. The above transformations are used to change all categorical variables we want to include in our model to number expressions so that SMOTE can be used to deal with the imbalanced data.

### Using SMOTE to handle inbalanced data

In [8]:
y = df3['f_purchase_lh']
X = df3.drop('f_purchase_lh', axis=1)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE to balance the classes
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

## Model building

### RandomForest

In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train_smote, y_train_smote, test_size=0.2, random_state=42)

# Create a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_rep)

# 5-fold cross validation
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

cv_scores = cross_val_score(rf_classifier, X_train_smote, y_train_smote, cv=5, scoring='accuracy')

print("Cross-Validation Scores:")
print(cv_scores)

Accuracy: 0.85

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.84      0.85      2802
           1       0.84      0.86      0.85      2732

    accuracy                           0.85      5534
   macro avg       0.85      0.85      0.85      5534
weighted avg       0.85      0.85      0.85      5534

Cross-Validation Scores:
[0.8230936  0.86194434 0.85833032 0.85902765 0.86101572]


## The cell below is **NOT** to be removed
##### The function is to be amended so that it accepts the given input (dataframe) and returns the required output (list). 
##### It is recommended to test the function out prior to submission
-------------------------------------------------------------------------------------------------------------------------------
##### The hidden_data parsed into the function below will have the same layout columns wise as the dataset *SENT* to you
##### Thus, ensure that steps taken to modify the initial dataset to fit into the model are also carried out in the function below

In [ ]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    '''DO NOT REMOVE THIS FUNCTION.

The function accepts a dataframe as input and return an iterable (list)
of binary classes as output.

The function should be coded to test on hidden data
and should include any preprocessing functions needed for your model to perform. 
    
All relevant code MUST be included in this function.'''
    result = [] 
    return result

##### Cell to check testing_hidden_data function

In [13]:
# This cell should output a list of predictions.
test_df = pd.read_parquet(filepath)
test_df = test_df.drop(columns=["f_purchase_lh"])
print(testing_hidden_data(test_df))

NameError: name 'filepath' is not defined

### Please have the filename renamed and ensure that it can be run with the requirements above being met. All the best!